<a href="https://colab.research.google.com/github/fireholder/Hung-yi-Lee-Machine-Learning-2020-Homework/blob/main/Note-7/hw7_Network_Pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 7 - Network Compression (Network Pruning)

> Author: Arvin Liu (b05902127@ntu.edu.tw)

若有任何問題，歡迎來信至助教信箱 ntu-ml-2020spring-ta@googlegroups.com

In [ ]:
# Download dataset
!gdown --id '19CzXudqN58R3D-1G8KeFWk8UDQwlb8is' --output food-11.zip
# Unzip the files
!unzip food-11.zip

  inflating: food-11/validation/9_84.jpg  
  inflating: food-11/validation/9_90.jpg  
  inflating: food-11/validation/0_280.jpg  
  inflating: food-11/validation/2_150.jpg  
  inflating: food-11/validation/7_95.jpg  
  inflating: food-11/validation/2_178.jpg  
  inflating: food-11/validation/7_81.jpg  
  inflating: food-11/validation/0_88.jpg  
  inflating: food-11/validation/7_80.jpg  
  inflating: food-11/validation/0_89.jpg  
  inflating: food-11/validation/2_179.jpg  
  inflating: food-11/validation/7_94.jpg  
  inflating: food-11/validation/2_151.jpg  
  inflating: food-11/validation/0_281.jpg  
  inflating: food-11/validation/9_91.jpg  
  inflating: food-11/validation/3_5.jpg  
  inflating: food-11/validation/9_85.jpg  
  inflating: food-11/validation/2_145.jpg  
  inflating: food-11/validation/0_295.jpg  
  inflating: food-11/validation/8_318.jpg  
  inflating: food-11/validation/7_43.jpg  
  inflating: food-11/validation/4_128.jpg  
  inflating: food-11/validation/10_222.jpg  


# Readme


HW7的任務是模型壓縮 - Neural Network Compression。

Compression有很多種門派，在這裡我們會介紹上課出現過的其中四種，分別是:

* 知識蒸餾 Knowledge Distillation
* 網路剪枝 Network Pruning
* 用少量參數來做CNN Architecture Design
* 參數量化 Weight Quantization

在這個notebook中我們會介紹Network Pruning，
而我們有提供已經做完Knowledge Distillation的小model來做Pruning。

* Model架構 / Architecute Design在同目錄中的hw7_Architecture_Design.ipynb。
* 下載已經train好的小model(0.99M): https://drive.google.com/open?id=12wtIa0WVRcpboQzhgRUJOpcXe23tgWUL
  * 參數為 base=16, width_mult=1 (default)

In [ ]:
import torch
import os
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
# Load進我們的Model架構(在hw7_Architecture_Design.ipynb內)
!gdown --id '1lJS0ApIyi7eZ2b3GMyGxjPShI8jXM2UC' --output "hw7_Architecture_Design.ipynb"
%run "hw7_Architecture_Design.ipynb"

Downloading...
From: https://drive.google.com/uc?id=1lJS0ApIyi7eZ2b3GMyGxjPShI8jXM2UC
To: /home/jovyan/work/Hung-yi-Lee-Machine-Learning-2020-Homework/Note-7/hw7_Architecture_Design.ipynb
100%|██████████████████████████████████████| 8.78k/8.78k [00:00<00:00, 11.6MB/s]


Network Pruning
===
在這裡我們會教Neuron Pruning。
<img src="https://i.imgur.com/Iwp90Wp.png" width="500px">

簡單上來說就是讓一個已經學完的model中的neuron進行刪減，讓整個網路變得更瘦。

## Weight & Neuron Pruning
* weight和neuron pruning差別在於prune掉一個neuron就等於是把一個matrix的整個column全部砍掉。但如此一來速度就會比較快。因為neuron pruning後matrix整體變小，但weight pruning大小不變，只是有很多空洞。

## What to Prune?
* 既然要Neuron Pruning，那就必須要先衡量Neuron的重要性。衡量完所有的Neuron後，就可以把比較不重要的Neuron刪減掉。
* 在這裡我們介紹一個很簡單可以衡量Neuron重要性的方法 - 就是看batchnorm layer的$\gamma$因子來決定neuron的重要性。 (by paper - Network Slimming)
  ![](https://i.imgur.com/JVpCm2r.png)
* 相信大家看這個pytorch提供的batchnorm公式應該就可以意識到為甚麼$\gamma$可以當作重要性來衡量了:)

* Network Slimming其實步驟沒有這麼簡單，有興趣的同學可以check以下連結。[Netowrk Slimming](https://arxiv.org/abs/1708.06519)


## 為甚麼這會 work?
* 樹多必有枯枝，有些neuron只是在躺分，所以有他沒他沒有差。
* 困難的說可以回想起老師說過的大樂透假說(The Lottery Ticket Hypothesis)就可以知道了。

## 要怎麼實作?
* 為了避免複雜的操作，我們會將StudentNet(width_mult=$\alpha$)的neuron經過篩選後移植到StudentNet(width_mult=$\beta$)。($\alpha > \beta$)
* 篩選的方法也很簡單，只需要抓出每一個block的batchnorm的$\gamma$即可。

## 一些實作細節
* 假設model中間兩層是這樣的:

|Layer|Output # of Channels|
|-|-|
|Input|in_chs|
|Depthwise(in_chs)|in_chs|
|BatchNorm(in_chs)|in_chs|
|Pointwise(in_chs, **mid_chs**)|**mid_chs**|
|**Depthwise(mid_chs)**|**mid_chs**|
|**BatchNorm(mid_chs)**|**mid_chs**|
|Pointwise(**mid_chs**, out_chs)|out_chs|

則你會發現利用第二個BatchNorm來做篩選的時候，跟他的Neuron有直接關係的是該層的Depthwise&Pointwise以及上層的Pointwise。
因此再做neuron篩選時記得要將這四個(包括自己, bn)也要同時prune掉。

* 在Design Architecure內，model的一個block，名稱所對應的Weight；

|#|name|meaning|code|weight shape|
|-|-|-|-|-|
|0|cnn.{i}.0|Depthwise Convolution Layer|nn.Conv2d(x, x, 3, 1, 1, group=x)|(x, 1, 3, 3)|
|1|cnn.{i}.1|Batch Normalization|nn.BatchNorm2d(x)|(x)|
|2||ReLU6|nn.ReLU6||
|3|cnn.{i}.3|Pointwise Convolution Layer|nn.Conv2d(x, y, 1),|(y, x, 1, 1)|
|4||MaxPooling|nn.MaxPool2d(2, 2, 0)||

# 评估哪些需要被prune
## 1. eval by weight
### 1.1 calculate sum of L1 norm
#### 1.2 FPGM
* filter pruning via geometric media
* 找出几何中心, prune 其他的

## 2. eval by BN's
### 2.1 network slimming
* 判断 $\gamma$ 的大小
* 如果不限制$\gamma$, 就会很难prune
* 对$\gamma$做regularization, 加上L1-penalty
## 3. eval by 0s after ReLU
### 3.1 APoZ
* Average percentage of zeros


In [ ]:
def network_slimming(old_model, new_model):
    params = old_model.state_dict()
    new_params = new_model.state_dict()
    
    # selected_idx: 每一層所選擇的neuron index
    selected_idx = []
    # 我們總共有7層CNN，因此逐一抓取選擇的neuron index們。
    for i in range(8):
        # 根據上表，我們要抓的gamma係數在cnn.{i}.1.weight內。
        importance = params[f'cnn.{i}.1.weight']
        # 抓取總共要篩選幾個neuron。
        old_dim = len(importance)
        new_dim = len(new_params[f'cnn.{i}.1.weight'])
        # 以Ranking做Index排序，較大的會在前面(descending=True)。
        ranking = torch.argsort(importance, descending=True)
        # 把篩選結果放入selected_idx中。
        selected_idx.append(ranking[:new_dim])

    now_processed = 1
    for (name, p1), (name2, p2) in zip(params.items(), new_params.items()):
        # 如果是cnn層，則移植參數。
        # 如果是FC層，或是該參數只有一個數字(例如batchnorm的tracenum等等資訊)，那麼就直接複製。
        if name.startswith('cnn') and p1.size() != torch.Size([]) and now_processed != len(selected_idx):
            # 當處理到Pointwise的weight時，讓now_processed+1，表示該層的移植已經完成。
            if name.startswith(f'cnn.{now_processed}.3'):
                now_processed += 1

            # 如果是pointwise，weight會被上一層的pruning和下一層的pruning所影響，因此需要特判。
            if name.endswith('3.weight'):
                # 如果是最後一層cnn，則輸出的neuron不需要prune掉。
                if len(selected_idx) == now_processed:
                    new_params[name] = p1[:,selected_idx[now_processed-1]]
                # 反之，就依照上層和下層所選擇的index進行移植。
                # 這裡需要注意的是Conv2d(x,y,1)的weight shape是(y,x,1,1)，順序是反的。
                else:
                    new_params[name] = p1[selected_idx[now_processed]][:,selected_idx[now_processed-1]]
            else:
                new_params[name] = p1[selected_idx[now_processed]]
        else:
            new_params[name] = p1

    # 讓新model load進被我們篩選過的parameters，並回傳new_model。        
    new_model.load_state_dict(new_params)
    return new_model


# Data Processing

我們的Dataset使用的是跟Hw3 - CNN同樣的Dataset，因此這個區塊的Augmentation / Read Image大家參考就好。

如果有不會的話可以回去看Hw3的colab。

In [ ]:
import re
import torch
from glob import glob
from PIL import Image
import torchvision.transforms as transforms

class MyDataset(torch.utils.data.Dataset):

    def __init__(self, folderName, transform=None):
        self.transform = transform
        self.data = []
        self.label = []

        for img_path in sorted(glob(folderName + '/*.jpg')):
            try:
                # Get classIdx by parsing image path
                class_idx = int(re.findall(re.compile(r'\d+'), img_path)[1])
            except:
                # if inference mode (there's no answer), class_idx default 0
                class_idx = 0
 
            image = Image.open(img_path)
            # Get File Descriptor
            image_fp = image.fp
            image.load()
            # Close File Descriptor (or it'll reach OPEN_MAX)
            image_fp.close()

            self.data.append(image)
            self.label.append(class_idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, self.label[idx]


trainTransform = transforms.Compose([
    transforms.RandomCrop(256, pad_if_needed=True, padding_mode='symmetric'),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])
testTransform = transforms.Compose([
    transforms.CenterCrop(256),
    transforms.ToTensor(),
])

def get_dataloader(mode='training', batch_size=32):

    assert mode in ['training', 'testing', 'validation']

    dataset = MyDataset(
        f'./food-11/{mode}',
        transform=trainTransform if mode == 'training' else testTransform)

    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=(mode == 'training'))

    return dataloader


# Pre-processing

我們已經提供原始小model binary，架構是hw7_Architecture_Design.ipynb中的StudentNet。

In [ ]:
# get dataloader
train_dataloader = get_dataloader('training', batch_size=32)
valid_dataloader = get_dataloader('validation', batch_size=32)

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
!gdown --id '12wtIa0WVRcpboQzhgRUJOpcXe23tgWUL' --output student_custom_small.bin

net = StudentNet().cuda()
net.load_state_dict(torch.load('student_custom_small.bin'))

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=1e-3)

Downloading...
From: https://drive.google.com/uc?id=12wtIa0WVRcpboQzhgRUJOpcXe23tgWUL
To: /content/student_custom_small.bin
100% 1.05M/1.05M [00:00<00:00, 67.1MB/s]


# Start Training

* 每次Prune rate是0.95，Prune完後會重新fine-tune 3 epochs。
* 其餘的步驟與你在做Hw3 - CNN的時候一樣。



In [ ]:
def run_epoch(dataloader, update=True, alpha=0.5):
    total_num, total_hit, total_loss = 0, 0, 0
    for now_step, batch_data in enumerate(dataloader):
        # 清空 optimizer
        optimizer.zero_grad()
        # 處理 input
        inputs, labels = batch_data
        inputs = inputs.cuda()
        labels = labels.cuda()
  
        logits = net(inputs)
        loss = criterion(logits, labels)
        if update:
            loss.backward()
            optimizer.step()

        total_hit += torch.sum(torch.argmax(logits, dim=1) == labels).item()
        total_num += len(inputs)
        total_loss += loss.item() * len(inputs)

    return total_loss / total_num, total_hit / total_num

now_width_mult = 1
for i in range(5):
    now_width_mult *= 0.95
    new_net = StudentNet(width_mult=now_width_mult).cuda()
    params = net.state_dict()
    net = network_slimming(net, new_net)
    now_best_acc = 0
    for epoch in range(5):
        net.train()
        train_loss, train_acc = run_epoch(train_dataloader, update=True)
        net.eval()
        valid_loss, valid_acc = run_epoch(valid_dataloader, update=False)
        # 在每個width_mult的情況下，存下最好的model。
        if valid_acc > now_best_acc:
            now_best_acc = valid_acc
            torch.save(net.state_dict(), f'custom_small_rate_{now_width_mult}.bin')
        print('rate {:6.4f} epoch {:>3d}: train loss: {:6.4f}, acc {:6.4f} valid loss: {:6.4f}, acc {:6.4f}'.format(now_width_mult, 
            epoch, train_loss, train_acc, valid_loss, valid_acc))


rate 0.9500 epoch   0: train loss: 0.4825, acc 0.8676 valid loss: 1.1327, acc 0.7971
rate 0.9500 epoch   1: train loss: 0.4858, acc 0.8656 valid loss: 1.1253, acc 0.7965
rate 0.9500 epoch   2: train loss: 0.4980, acc 0.8648 valid loss: 1.0623, acc 0.8003
rate 0.9500 epoch   3: train loss: 0.4837, acc 0.8726 valid loss: 1.1127, acc 0.8015
rate 0.9500 epoch   4: train loss: 0.5124, acc 0.8643 valid loss: 1.1098, acc 0.7974
rate 0.9025 epoch   0: train loss: 0.5723, acc 0.8444 valid loss: 1.2317, acc 0.7790
rate 0.9025 epoch   1: train loss: 0.5853, acc 0.8424 valid loss: 1.1912, acc 0.7851
rate 0.9025 epoch   2: train loss: 0.5930, acc 0.8392 valid loss: 1.1866, acc 0.7822
rate 0.9025 epoch   3: train loss: 0.5724, acc 0.8439 valid loss: 1.1844, acc 0.7799
rate 0.9025 epoch   4: train loss: 0.6047, acc 0.8408 valid loss: 1.1769, acc 0.7808
rate 0.8574 epoch   0: train loss: 0.7062, acc 0.8101 valid loss: 1.2120, acc 0.7671
rate 0.8574 epoch   1: train loss: 0.6902, acc 0.8100 valid loss:

# Inference

同Hw3，請參考該作業:)。

# Q&A

有任何問題Network Compression的問題可以寄信到b05902127@ntu.edu.tw。

時間允許的話我會更新在這裡。